## Video Demos

### Install av library

In [ ]:
!conda install av -c conda-forge -y

### Download and play video

In [ ]:
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Intro.mp4 -o video.mp4

In [ ]:
%%HTML
<video width="320" height="240" controls>
  <source src="video.mp4" type="video/mp4">
</video>

### Count frames and show the 25th one

In [ ]:
import av
container = av.open('video.mp4')

%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

for frame in container.decode(video=0):
    if (frame.index == 25):
        img = frame.to_image()
        imshow(img)
frameCount = frame.index - 1
print(str(frameCount) + " frames")

## Using the Video Indexer API

### Create a Video Indexer account
https://www.videoindexer.ai/ 

In [ ]:
viLocation = 'trial'
viAccountID = 'YOUR_ACCOUNT_ID'

### Create a Video Indexer API Subscription
https://api-portal.videoindexer.ai/

In [ ]:
viKey = 'YOUR_PRIMARY_KEY'

### Get Account-level Access Token

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': viKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'allowEdit': True,
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("GET", "/auth/%s/Accounts/%s/AccessToken?%s" % (viLocation, viAccountID, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    accessToken = data.decode("UTF-8").replace('"', '')
    print("accessToken: " + accessToken)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


### Upload a Video for Processing


In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json

# We'll upload this video from GitHub to the Video Indexer
video_url='https://github.com/MicrosoftLearning/AI-Introduction/raw/master/files/vid.mp4'

headers = {
    # Request headers
    'Content-Type': 'multipart/form-data',
}

params = urllib.parse.urlencode({
    # Request parameters
    'accessToken' : accessToken,
    'name': 'vid',
    'privacy': 'Private',
    'videoUrl': video_url,
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("POST", "/%s/Accounts/%s/Videos?%s" % (viLocation, viAccountID, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    parsed = json.loads(data.decode("UTF-8"))
    print(parsed)
    conn.close()
except Exception as e:
    print(e)

### Get Video ID

In [ ]:
videoID = parsed['id']
print("videoId: " + videoID)

### Check Processing Status

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
}

params = urllib.parse.urlencode({
    # Request parameters
    'accessToken': accessToken,
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("GET", "/%s/Accounts/%s/Videos/%s/Index?%s" % (viLocation, viAccountID, videoID, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    parsed = json.loads(data.decode("UTF-8"))
    conn.close()
except Exception as e:
    print(e)

print("State : " + parsed['state'])

### Get Video-level Access Token


In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': viKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'allowEdit': True,
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("GET", "/auth/%s/Accounts/%s/Videos/%s/AccessToken?%s" % (viLocation, viAccountID, videoID, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    videoAccessToken = data.decode("UTF-8").replace('"', '')
    print("videoAccessToken: " + videoAccessToken)
    conn.close()
except Exception as e:
    print(e)

### View the video

In [ ]:
from IPython.core.display import HTML

playerUrl = "https://www.videoindexer.ai/embed/player/{0}/{1}/?accessToken={2}".format(viAccountID,videoID,videoAccessToken)

HTML('<iframe width=900 height=600 src="%s"/>' % playerUrl )

### View the Video Breakdown

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
}

params = urllib.parse.urlencode({
    # Request parameters
    'accessToken': videoAccessToken,
    'language': 'English',
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("GET", "/%s/Accounts/%s/Videos/%s/Index?%s" % (viLocation, viAccountID, videoID, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    strData = data.decode("UTF-8")
    jData = json.loads(strData)
    print (json.dumps(jData, sort_keys=True, indent=2))
    conn.close()
except Exception as e:
    print(e)


### Get Details of Faces Identified in the Video

In [ ]:
print(json.dumps(jData["summarizedInsights"]["faces"], sort_keys=True, indent=2))

### View a Face Thumbnail

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
from PIL import Image
from io import BytesIO
from matplotlib.pyplot import imshow
%matplotlib inline

thumbnailId = jData["summarizedInsights"]["faces"][0]["thumbnailId"]

headers = {
}

params = urllib.parse.urlencode({
    # Request parameters
    'accessToken': videoAccessToken,
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("GET", "/%s/Accounts/%s/Videos/%s/Thumbnails/%s?%s" % (viLocation, viAccountID, videoID, thumbnailId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    img = Image.open(BytesIO(data))
    imshow(img)
    conn.close()
except Exception as e:
    print(e)

### View (and Edit) People Insights


In [ ]:
from IPython.core.display import HTML

insightsUrl = "https://www.videoindexer.ai/embed/insights/{0}/{1}/?accessToken={2}".format(viAccountID,videoID,videoAccessToken)

HTML('<iframe width=900 height=600 src="%s"/>' % insightsUrl )

### Reload Breakdown and Check Updated Face Details


In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
}

params = urllib.parse.urlencode({
    # Request parameters
    'accessToken': videoAccessToken,
    'language': 'English',
})

try:
    conn = http.client.HTTPSConnection('api.videoindexer.ai')
    conn.request("GET", "/%s/Accounts/%s/Videos/%s/Index?%s" % (viLocation, viAccountID, videoID, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    strData = data.decode("UTF-8")
    jData = json.loads(strData)
    print(json.dumps(jData["summarizedInsights"]["faces"], sort_keys=True, indent=2))
    conn.close()
except Exception as e:
    print(e)


The name you specified has been assigned to this person, and if this face is detected in any future videos, it will be identified as the same person in this video. 